## Working, but ugly :(

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
import metal
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
from dataset import BERTDataset

In [6]:
model = 'bert-base-uncased' # also try bert-base-multilingual-cased (recommended)
src_path = os.path.join(os.environ['GLUEDATA'], 'SST-2/{}.tsv')
dataloaders = {}
for split in ['train','dev']:
    dataset = BERTDataset(
        src_path.format(split),
        sent1_idx=0,
        label_idx=1,
        skip_rows=1,
        label_fn=lambda x: int(x)+1 # labels are scores [1, 2] (multiclass with cardinality k)
    )

    dataloaders[split] = dataset.get_dataloader(batch_size=32)

In [7]:
import torch.nn as nn
from metal.end_model import EndModel

class BertEncoder(nn.Module):
    def __init__(self):
        super(BertEncoder, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

    def forward(self, data):
        tokens, segments, mask = data
        # TODO: check if we should return all layers or just last hidden representation 
        _, hidden_layer = self.bert_model(tokens, segments, mask)
        return hidden_layer

class SSTHead(EndModel):     
    def __init__(self, output_dims, **kwargs):
        super(SSTHead, self).__init__(output_dims, **kwargs)
        # self.criteria = nn.modules.loss.MSELoss()
    """
    def _loss(self, data, Y):
        output = self.forward(data)
        prediction = torch.sigmoid(output)
        return self.criteria(prediction, Y)

    def _get_loss_fn(self):
        # This self.preprocess_Y allows us to not handle preprocessing
        # in a custom dataloader, but decreases speed a bit
        return self._loss
    """

In [15]:
encoder_module = BertEncoder()
end_model = SSTHead(
    [768, 2],
    input_module=encoder_module,
    seed=123,
    device = torch.device("cuda"),
    skip_head=False,
    input_relu=False,
    input_batchnorm=False,
    verbose=False
)

In [16]:
end_model.train_model(dataloaders['train'], valid_data=dataloaders['dev'],
                      lr=0.01, l2=0.01, 
                      n_epochs=5, checkpoint_metric='train/loss',
                      log_valid_metrics=["train/loss", "valid/loss"],
                      checkpoint_metric_mode='min',
                      verbose=True, progress_bar=True
                    )



  0%|          | 0/2105 [00:00<?, ?it/s]

Could not find kwarg "valid_data" in destination dict.
Could not find kwarg "checkpoint_metric" in destination dict.
Could not find kwarg "log_valid_metrics" in destination dict.
Could not find kwarg "checkpoint_metric_mode" in destination dict.
Could not find kwarg "progress_bar" in destination dict.




  0%|          | 0/2105 [00:04<?, ?it/s, avg_loss=7.16]

  0%|          | 1/2105 [00:04<2:53:11,  4.94s/it, avg_loss=7.16]

  0%|          | 1/2105 [00:07<2:53:11,  4.94s/it, avg_loss=18.7]

  0%|          | 2/2105 [00:07<2:33:12,  4.37s/it, avg_loss=18.7]

  0%|          | 2/2105 [00:11<2:33:12,  4.37s/it, avg_loss=28.1]

  0%|          | 3/2105 [00:11<2:24:28,  4.12s/it, avg_loss=28.1]

  0%|          | 3/2105 [00:15<2:24:28,  4.12s/it, avg_loss=23.7]

  0%|          | 4/2105 [00:15<2:17:57,  3.94s/it, avg_loss=23.7]

  0%|          | 4/2105 [00:17<2:17:57,  3.94s/it, avg_loss=26.8]

  0%|          | 5/2105 [00:17<2:03:08,  3.52s/it, avg_loss=26.8]

  0%|          | 5/2105 [00:21<2:03:08,  3.52s/it, avg_loss=25.2]

  0%|          | 6/2105 [00:21<2:08:11,  3.66s/it, avg_loss=25.2]

  0%|          | 6/2105 [00:26<2:08:11,  3.66s/it, avg_loss=23.5]

  0%|          | 7/2105 [00:26<2:22:19,  4.07s/it, avg_loss=23.5]

  0%|          | 7/2105 [00:29<2:22:19,  4.07s/it, avg_loss=21.8]

  0

KeyboardInterrupt: 